In [2]:
# to test:
# 1. bake multi-search using example in a sinple agent
# 2. build a researcher agent

useful links:

https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/get-token-count

#### **simple response**

In [1]:
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

def generate_function_call(prompt: str) -> str:
    # Load the Vertex AI Gemini API to use function calling
    model = GenerativeModel("gemini-pro")

    # Specify a function declaration and parameters for an API request
    get_current_weather_func = FunctionDeclaration(
        name="get_current_weather",
        description="Get the current weather in a given location",
        # Function parameters are specified in OpenAPI JSON schema format
        parameters={
            "type": "object",
            "properties": {"location": {"type": "string", "description": "Location"}},
        },
    )

    # Define a tool that includes the above get_current_weather_func
    weather_tool = Tool(
        function_declarations=[get_current_weather_func],
    )

    # Prompt to ask the model about weather, which will invoke the Tool
    prompt = prompt

    # Instruct the model to generate content using the Tool that you just created:
    response = model.generate_content(
        prompt,
        generation_config={"temperature": 0},
        tools=[weather_tool],
    )

    # Transform the structured data into a Python dictionary
    params = {}
    for key, value in response.candidates[0].content.parts[0].function_call.args.items():
        params[key] = value
    params

    # This is where you would make an API request to get the location of the store closest to the user.
    # Here we'll use synthetic data to simulate a response payload from an external API.
    api_response = """{ "location": "Boston, MA", "temperature": 38, "description": "Partly Cloudy",
                  "icon": "partly-cloudy", "humidity": 65, "wind": { "speed": 10, "direction": "NW" } }"""

    # Return the API response to Gemini so it can generate a model response or request another function call
    response = model.generate_content(
        [
        Content(role="user", parts=[
            Part.from_text(prompt),
        ]),
        Content(role="function", parts=[
            Part.from_dict({
                "function_call": {
                    "name": "get_current_weather",
                }
            })
        ]),
        Content(role="function", parts=[
            Part.from_function_response(
                name="get_current_weather",
                response={
                    "content": api_response,
                }
            )
        ]),
        ],
        tools=[weather_tool],
    )

    answer = response.candidates[0].content.parts[0].text

    return (response, params, answer)

In [2]:
test = generate_function_call("what's the weather in delhi")

In [3]:
test[2]

"It's partly cloudy in delhi with a temperature of 38 degrees. The humidity is 65% and the wind speed is 10mph in the NW direction."

In [4]:
from vectorengine import VectorEngine
ecsop = VectorEngine(folder_path="docs/general-docs", collection_name="sop")

#### **multi-turn**

In [5]:
from vertexai import generative_models
from vertexai.generative_models import GenerativeModel
model = GenerativeModel("gemini-pro")

search_knowledge_func = FunctionDeclaration(
    name="search_knowledge",
    description="to answer any question related to trade settlement, tennis players, and countries information like tourism websites",
    parameters={
    "type": "object",
    "properties": {
        "query": {
            "type": "string",
            "description": "your rephrased search query to retrieve knowledge from a vector database"
        },
    },
         "required": [
            "query"
      ]
  },
)

search_tool = generative_models.Tool(
  function_declarations=[search_knowledge_func],
)



In [6]:
RESEARCHER = """
You are ResearchGPT, a helpful assistant who uses a given list of sources to take notes in order to answer a question. Your notes
would be passed to a question generator which would generate additional questions to answer the question in full if there is not
enough information in the notepad. 
Hints:
1. Do not comment, just diligently take notes
2. Extract a list of all the facts directly related to the question concisely onto your notepad.
"""

QUESTION_GENERATOR = """
You are QuestionsGPT, a helpful assistant who looks at the current state of acquired knowledge
and intelligently generates an additional question and query to search a knowledge base.
Here are some tips;
1. Create questions and queries which can concisely and dircetly answer the user's primary query
Provide your answer in JSON structure like this {"enough_info": "is there enough info in the notepad to anser this question. allowed values: yes/no", "additional_query": "your search query to get additional info to answer user question if not enough info else 'na'"}
"""

FINAL_ANSWERER = """
Using the given information in a notepad from a researcher, concisely answer the user question.
"""

GENERAL_ASSISTANT = """
You are Sage, a friendly and helpful agent who helps users using the given functions.
Here are the rules you have to strictly adhere to:
1. Don't make assumptions about what values to plug into functions. 
2. You will use the search_knowledge tool to find relavent knowlege articles to create the answer.
3. Being smart in your research, if the search does not come back with the enough information, rephrase the question and try again.
4. Review the result of the search and use it to guide your next search if needed.
5. If the question is complex, break down to smaller search steps and find the answer in multiple steps.
6. Answer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
"""

In [7]:
api_response_1 = """{ "content" : "tennis player polanski was born in Serbia" }"""
api_response_2 = """{ "content" : "capital of Serbia is Belgrade" }"""

In [8]:
# understand proto formatting
messages = [
Content(role="user", parts=[
    Part.from_text("what's the capital of the place where tennis player polanski born"),
]),
Content(role="function", parts=[
    Part.from_dict({
        "function_call": {
            "name": "search_knowledge",
        }
    })
]),
Content(role="function", parts=[
    Part.from_function_response(
        name="search_knowledge",
        response={
            "content": api_response_1,
        }
    )
]),
Content(role="function", parts=[
    Part.from_dict({
        "function_call": {
            "name": "search_knowledge",
        }
    })
]),
Content(role="function", parts=[
    Part.from_function_response(
        name="search_knowledge",
        response={
            "content": api_response_2,
        }
    )
]),
]


In [9]:
messages

[role: "user"
 parts {
   text: "what\'s the capital of the place where tennis player polanski born"
 },
 role: "function"
 parts {
   function_call {
     name: "search_knowledge"
   }
 },
 role: "function"
 parts {
   function_response {
     name: "search_knowledge"
     response {
       fields {
         key: "content"
         value {
           string_value: "{ \"content\" : \"tennis player polanski was born in Serbia\" }"
         }
       }
     }
   }
 },
 role: "function"
 parts {
   function_call {
     name: "search_knowledge"
   }
 },
 role: "function"
 parts {
   function_response {
     name: "search_knowledge"
     response {
       fields {
         key: "content"
         value {
           string_value: "{ \"content\" : \"capital of Serbia is Belgrade\" }"
         }
       }
     }
   }
 }]

In [10]:
response = model.generate_content(
    messages,
    tools=[search_tool],
)

In [238]:
def generate_qns(query, notepad):
    notepad_example = """
    - Polanski was born in Serbia and started playing tennis at 4 years old.
    - Turned professional at age 16 after an impressive junior career.
    - His playing style includes offensive baseline play and quick net skills, known for a signature serve.
    - Has reached semifinals of several ATP 250 events and has victories over higher-ranked opponents.
    - Polanski runs tennis clinics for underprivileged children in his home country.
    - Experts predict he will continue to rise in the ranks of professional tennis.
    """
    example_query1 = "where was Polanski born?"
    example_query2 = "what is the capital of the place where Polanski was born?"
    messages = []
    messages.append({"role": "system", "content": f"{QUESTION_GENERATOR}"})
    messages.append({"role": "user", "content": f"Is this enough to answer the user's initial question: {example_query1}\n Here's the notepad: {notepad_example}\n What additional question has to generated?"})
    messages.append({"role": "assistant", "content": '{"enough_info": "yes", "additional_query": "na"}'})
    messages.append({"role": "user", "content": f"Is this enough to answer the user's initial question: {example_query2}\n Here's the notepad: {notepad_example}\n What additional question has to generated?"})
    messages.append({"role": "assistant", "content": '{"enough_info": "no", "additional_query": "current capital of Serbia"}'})
    messages.append({"role": "user", "content": f"Is this enough to answer the user's initial question: {query}\n Here's the notepad we have: {notepad}\n What additional question has to generated?"}) 
    chat_response = chat_completion_request_json(
        messages
    )
    # generate json response
    assistant_message = chat_response.choices[0].message.content
    return assistant_message


def final_answer(query, notepad):
    example_query1 = "where was Polanski born?"
    example_query2 = "what is the capital of the place where Polanski was born?"
    messages = []
    messages.append({"role": "system", "content": f"{FINAL_ANSWERER}"})
    messages.append({"role": "user", "content": f"query: {query}\n Here's the notepad we have for your reference: {notepad}"}) 
    chat_response = chat_completion_request(
        messages
    )
    # generate json response
    assistant_message = chat_response.choices[0].message.content
    return assistant_message

In [239]:
def call_researcher(query, articles, n):
    notepad = ""
    user_query = query
    enough_info = 'no'
    i = 0

    while enough_info.lower() == 'no':
        messages=[]
        search_results = ecsop.query(query, n) 
        #print(search_results)
        if not messages:
            messages.append({"role": "system", "content": f"{RESEARCHER}"})

        messages.append({"role": "user", "content": f":these are your sources {search_results} \nExtract only the relevant information to this query: {query}\n into your notepad."})
        
        chat_response = chat_completion_request(
            messages
        )
        assistant_message = chat_response.choices[0].message.content
        notepad += assistant_message
        print(f"\n{notepad}")
        generated_qns = generate_qns(user_query, notepad)
        additional_qns = json.loads(generated_qns)
        enough_info = additional_qns['enough_info']
        query = additional_qns['additional_query']
        print(enough_info)
        print(query)
        i += 1
        if i == 3:
            break

    answer = final_answer(user_query, notepad)
    return answer

In [240]:
function_calling_chat = model.start_chat(response_validation=False)
model_response = function_calling_chat.send_message(f"{GENERAL_ASSISTANT}", tools=[search_tool])
print("model_response\n", model_response)

model_response
 candidates {
  content {
    role: "model"
    parts {
      text: "What are some interesting facts about trade settlements?"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 235
  candidates_token_count: 9
  total_token_count: 244
}



In [241]:
function_calling_chat = model.start_chat()
model_response = function_calling_chat.send_message("what's the tourism website of Serbia", tools=[search_tool])
print("model_response\n", model_response)

model_response
 candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "search_knowledge"
        args {
          fields {
            key: "query"
            value {
              string_value: "what\'s the tourism website of Serbia"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 44
  candidates_token_count: 12
  total_token_count: 56
}



In [242]:
arguments = model_response.candidates[0].content.parts[0].function_call.args
print(arguments["query"])

what's the tourism website of Serbia


In [243]:
# model_response = function_calling_chat.send_message("in boston please?", tools=[weather_tool])
# print("model_response\n", model_response)

In [244]:
from IPython.display import display, Markdown
#Markdown(ecsop.query(arguments["query"], 1))

In [245]:
model_response = function_calling_chat.send_message(
  Part.from_function_response(
      name="search_knowledge",
      response={
          "content": {"content": ecsop.query(arguments["query"], 1)},
      }
  ),
  tools=[search_tool]
)
print("model_response\n", model_response)

model_response
 candidates {
  content {
    role: "model"
    parts {
      text: "The tourism website of Serbia is http://www.serbia.travel"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 620
  candidates_token_count: 14
  total_token_count: 634
}



In [246]:
arguments = model_response.candidates[0].content.parts[0].function_call.args
if arguments:
    print(arguments["query"])

In [247]:
if arguments:
  model_response = function_calling_chat.send_message(
    Part.from_function_response(
        name="search_knowledge",
        response={
            "content": {"content": ecsop.query(arguments["query"], 1)},
        }
    ),
    tools=[weather_tool]
  )
  print("model_response\n", model_response)

In [248]:
function_calling_chat.history

[role: "user"
 parts {
   text: "what\'s the tourism website of Serbia"
 },
 role: "model"
 parts {
   function_call {
     name: "search_knowledge"
     args {
       fields {
         key: "query"
         value {
           string_value: "what\'s the tourism website of Serbia"
         }
       }
     }
   }
 },
 role: "user"
 parts {
   function_response {
     name: "search_knowledge"
     response {
       fields {
         key: "content"
         value {
           struct_value {
             fields {
               key: "content"
               value {
                 string_value: "title:\nserbia\n\ncontent:\n# Serbia\n\n![Serbia Landscape](URL_to_an_image_of_Serbia_landscape)\n\n## Overview\n\nSerbia is a landlocked country situated in Southeast Europe, in the central and western part of the Balkan Peninsula. It shares its borders with Hungary to the north, Romania to the northeast, Bulgaria to the southeast, North Macedonia to the south, Croatia and Bosnia and Herzegovina t